In [1]:
import requests
import json
import chess.pgn
import io
import pandas as pd
import math
import numpy as np

In [2]:
USERNAME = "branbra"
url = f"https://api.chess.com/pub/player/" + USERNAME + "/games/2022/09"
data = requests.get(url)
if data.status_code != 200:
    raise Exception("The following response was returned: " + str(data.status_code))
else:
    data = json.loads(data.text)
    games = data["games"]
all_games=[]
for game in games:
    pgn = (game['pgn'])
    pgn = io.StringIO(pgn)
    game = chess.pgn.read_game(pgn)
    all_games.append(game)

In [3]:
game_list = []
for g in all_games:
    moves = (g.mainline_moves())
    moves = [str(x) for x in moves]
    
    white = (g.headers['White'])
    if white.lower() == "branbra":
        playing_as_white = 1
    else:
        playing_as_white = 0
    
    if len(moves)>1:
        move_made = (moves[1])
    else:
        move_made = ""
    
    game = {"date": (g.headers["Date"]), "player_white": white, "player_black": (g.headers['Black']), "playing_as_white" : playing_as_white, "result": (g.headers['Result']), "termination": (g.headers['Termination']), "moves": moves, "no_of_moves": (math.ceil(len(moves)/2)), "first_move": (moves[0]), "response": move_made}

    
    game_list.append(game)
game_list = pd.DataFrame(game_list)

In [6]:
class TreeNode:
    def __init__(self, value) -> None:
        self.value = value
        self.children = {}  # children[e2e4] = <TreeNode object>
        self.num_occur = 1
    def add_child(self, val):
        self.children[val] = TreeNode(val)
    def increment(self):
        self.num_occur += 1

In [9]:
starting_moves = {}  # FORMAT: starting_moves[e2e4] = TreeNode(value=e2e4)
for game in game_list["moves"]:
    if game[0] not in starting_moves:
        starting_moves[game[0]] = TreeNode(game[0])
    else:
        starting_moves[game[0]].increment()
    cur = starting_moves[game[0]]
    for i in range(1, len(game)):
        if game[i] not in cur.children:  # cur.children = [e7e5: TreeNode object, d7d5: TreeNode object]
            cur.add_child(game[i])
        else:
            cur.children[game[i]].increment()
        cur = cur.children[game[i]]
for move in starting_moves:
    print(starting_moves[move].num_occur)


98
317
3
5
6
4
6
1
2
